# Axis of interest

In [1]:
import os
import json
import lmstudio as lms
import sys
import re
from langchain_openai import ChatOpenAI
from pathlib import Path

repo_root = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path("/Users/jpconde/Facultad/tesis")
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

In [2]:
from axis_of_interest.schemas import list_of_aoi, indice_aoi
from axis_of_interest.prompts import template_prompt_generate_plot_schema, template_prompt_generate_cuento, tenplate_prompt_generar_axis_of_interest
from axis_of_interest.utils import render_aoi_md, render_plot_schema_md

### Crear AoI en python

Pasamos los AoI del Dr.Gervás a python. Usamos pydantic


In [3]:
for aoi in list_of_aoi:
    print(render_aoi_md(aoi))
    print("\n")


AXISofINTEREST =  CALL TO ACTION REWARD
PROTAGONIST =  called
ROLES =  called caller rewarded
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Call

  CallToAction  characters(called=Hero,caller=Sender)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Reward

  Reward  characters(rewarded=Hero)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  CONFLICT
PROTAGONIST =  attacker
ROLES =  attacker defender winner loser
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Struggle

  Struggle  characters(attacker=X,defender=Y)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Victory

  Victory  characters(winner=X,loser=Y)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  DONOR
PROTAGONIST =  tested
ROLES =  tested tester user gift
───────────────────────────────────────────────────────

### Crear Plot Schema

Le pegamos a una api de un LLM, con algunos axes of interest en el prompt y le pedimos que nos genere un Plot Schema

In [4]:
print(indice_aoi)

{'CALL TO ACTION REWARD': 0, 'CONFLICT': 1, 'DONOR': 2, 'INTERDICTION VIOLATED': 3, 'JOURNEY': 4, 'KIDNAPPING': 5, 'KIDNAPPING PUNISHMENT': 6, 'PURSUIT': 7, 'RAGS 2 RICHES': 8, 'RELENTING GUARDIAN': 9, 'REPENTANCE': 10, 'RIVALRY': 11, 'SHIFTING LOVE': 12, 'TASK': 13, 'UNRELENTING GUARDIAN': 14}


In [5]:
model1 = ChatOpenAI(
    model="gpt-5",
    temperature=0.0,
)

lista1 = [list_of_aoi[indice_aoi["RAGS 2 RICHES"]], list_of_aoi[indice_aoi["CONFLICT"]], list_of_aoi[indice_aoi["DONOR"]]]

prompt_generate_plot_schema = template_prompt_generate_plot_schema.replace(
    "{axis_of_interest}",
    json.dumps([a.model_dump() for a in lista1], indent=2)
)

result1 = model1.invoke(prompt_generate_plot_schema)

In [6]:
print(render_plot_schema_md(result1.content))

PLOT-SCHEMA =  RAGS2RICHES-DONOR-CONFLICT
DESCRIPTION:
Trama compuesta: un héroe en la miseria aspira a más, es probado por un donante, recibe y usa un amuleto, se transforma, enfrenta a un villano y alcanza la recompensa.
────────────────────────────────────────────────────────────────
AXISofINTEREST =  RAGS 2 RICHES
PLOT-SPAN-NAME =  Rags
DESCRIPTION:
El héroe sufre la pobreza y la carencia inicial.

Poverty  characters(sufferer=Hero)
  — Hero padece necesidad material y simbólica; punto de partida humilde.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  RAGS 2 RICHES
PLOT-SPAN-NAME =  Aspiration
DESCRIPTION:
El héroe formula un deseo explícito de superación.

Aspiration  characters(aspirer=Hero)
  — Hero fija una meta: salir de la miseria y ganar dignidad.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  DONOR
PLOT-SPAN-NAME =  Tested
DESCRIPTION:
El donante pone a prueba al héroe; si la supera, le concede un age

### Crear Cuento Completo

Dado un Plot Schema usamos un LLM para generar un cuento

In [7]:
prompt_generate_cuento = template_prompt_generate_cuento.replace(
    "{plot_schema}",
    result1.content
)

In [8]:
cuento = model1.invoke(prompt_generate_cuento)
print(cuento.content)

En los barrios bajos de San Velorio, donde las aceras eran puertos de sombras y los techos rezumaban la sal de un mar invisible, vivía Mateo, encuadernador ambulante. Cosía lomos con hilo rescatado y pegamento aguado, y las portadas, de cartón desparejo, parecían querer cerrarse para siempre sobre historias a las que faltaban páginas. Si el hambre tenía música, en la casa de Mateo sonaba a cucharas golpeando fondo vacío y a viento silbando por rendijas. Su madre, ya muerta, le había dejado una cajita con agujas y un consejo: “Ten manos firmes y corazón claro.” Lo primero lo aprendió a fuerza de frío; lo segundo, a fuerza de necesidad.

Cada mañana pasaba frente al Palacio de los Arcedianos, donde las ventanas reflejaban un sol dorado que en su barrio parecía de cobre gastado. Miraba aquellas losas brillantes y pensaba que el mundo era un libro con tapas que él no podía abrir. No envidiaba el oro; codiciaba, apenas, la dignidad de caminar por la plaza sin que lo rozara la sospecha del g

### Crear AoI dado un cuento

Acá lo que hacemos es el paso inverso. Queremos mandar un cuento y ver si el LLM es capaz de detectar los AoI. Hicimos dos experimentos:
- Con el cuento que creamos recien y ver si nos devuelve los AoI que usamos
- Con un cuento cualquiera, este cuento nosotros lo analizamos nosotros y detectamos algunos AoI

In [9]:
from langchain_core.prompts import PromptTemplate

template_prompt_devolver_aoi = PromptTemplate.from_template("Dado el siguiente cuento: {cuento}, y la siguiente lista de Axis of Interest: {axis_of_interest}, devuelve una lista con los nombres de los Axis of Interest que abarquen la historia. Tambien el orden en que se entrelazan los distintos Plot Spans.")
prompt_devolver_aoi = template_prompt_devolver_aoi.invoke({"cuento": cuento.content, "axis_of_interest": json.dumps([a.model_dump() for a in lista1], indent=2)})


In [10]:
aoi_devuelto = model1.invoke(prompt_devolver_aoi)

In [11]:
print(aoi_devuelto.content)

- Axis of Interest que abarquen la historia: 
  - RAGS 2 RICHES
  - CONFLICT
  - DONOR

- Orden entrelazado de los Plot Spans:
  1) RAGS 2 RICHES — Rags
  2) RAGS 2 RICHES — Aspiration
  3) DONOR — Tested (incluye la prueba, reacción y ProvisionOfAMagicalAgent)
  4) DONOR — UseOfAMagicalAgent (primeros usos cotidianos)
  5) RAGS 2 RICHES — Transformation (organización vecinal y voz firme)
  6) CONFLICT — Struggle (confrontación pública con Malvano)
  7) DONOR — UseOfAMagicalAgent (durante el enfrentamiento)
  8) CONFLICT — Victory (destitución de Malvano)
  9) RAGS 2 RICHES — Riches (taller, respeto, puesto en el Consejo; mentoría)


### Crear pipline que haga todo esto de una 

Dar una lista de AoI que quiero en mi historia -> crear plot schema -> crear cuento -> ver si el LLM predice bien los AoI que usó para crear la historia

In [23]:
def pipeline(lista_aoi, modelo):
    prompt_generar_plot_schema = template_prompt_generate_plot_schema.replace(
        "{axis_of_interest}",
        json.dumps([a.model_dump() for a in lista_aoi], indent=2)
    )
    plot_schema = modelo.invoke(prompt_generar_plot_schema)

    prompt_generar_cuento = template_prompt_generate_cuento.replace(
        "{plot_schema}",
        plot_schema.content
    )
    cuento = modelo.invoke(prompt_generar_cuento)

    prompt_devolver_aoi = template_prompt_devolver_aoi.invoke({"cuento": cuento.content, "axis_of_interest": json.dumps([a.model_dump() for a in lista_aoi], indent=2)})

    aoi_devuelto = modelo.invoke(prompt_devolver_aoi)
    return plot_schema, cuento, aoi_devuelto

In [24]:
aoi_prueba2 = [list_of_aoi[indice_aoi["JOURNEY"]], list_of_aoi[indice_aoi["RIVALRY"]], list_of_aoi[indice_aoi["CALL TO ACTION REWARD"]], list_of_aoi[indice_aoi["REPENTANCE"]], list_of_aoi[indice_aoi["PURSUIT"]]]
model2 = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.0,
)

In [25]:
ps2, cuento2, aoi2 = pipeline(aoi_prueba2, model2)

In [28]:
print(render_plot_schema_md(ps2.content))


PLOT-SCHEMA =  PLOT SCHEMA NEURO-SIMBÓLICO
DESCRIPTION:
Plot schema que agrupa los ejes narrativos JOURNEY, RIVALRY, CALL TO ACTION REWARD, REPENTANCE y PURSUIT, con sus respectivos spans y plot atoms, siguiendo un enfoque neuro-simbólico para cohesión y trazados de relaciones de largo alcance.
────────────────────────────────────────────────────────────────
AXISofINTEREST =  JOURNEY
PLOT-SPAN-NAME =  Out
DESCRIPTION:
Etapa 'Out' dentro del eje 'Journey'.

Departure  characters(traveller=Hero)
  — Personajes: traveller=Hero. Ubicaciones: origin=Home, destination=Elsewhere.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  JOURNEY
PLOT-SPAN-NAME =  Back
DESCRIPTION:
Etapa 'Back' dentro del eje 'Journey'.

Return  characters(traveller=Hero)
  — Personajes: traveller=Hero. Ubicaciones: origin=Elsewhere, destination=Home.

────────────────────────────────────────────────────────────────
AXISofINTEREST =  RIVALRY
PLOT-SPAN-NAME =  RivalryDeclared
DESCRIPTIO

In [29]:
print(cuento2.content)

Escena Out - Departure
El viajero conocido como Hero se despide de su hogar con una mezcla de temblor y curiosidad. La puerta golpea suavemente al cerrarse y, fuera, el mundo parece respirarlas en susurros: un mapa antiguo que late bajo la tela de la noche y una ruta que aún no entiende. Hero camina hacia Elsewhere, donde cada paso parece abrir un ojo nuevo en el paisaje. No lleva más que su vela interior y el hambre de saber qué hay al otro lado del horizonte. Mientras avanza, el origen se desvanece y la promesa de lo desconocido toma forma: la aventura ha comenzado.

Escena Back - Return
Horas o años —la memoria se confunde— después, el regreso es una rueda que vuelve a girar en la misma casa. Hero regresa a Home, no porque haya encontrado todas las respuestas, sino porque el viaje ha sembrado preguntas que sólo la tierra de su hogar puede hacer germinar. Las paredes conocen su nombre y el silencio entre las paredes parece respirar con él, como si el mundo entero hubiera aprendido a 

In [30]:
print(aoi2.content)

A continuación:

- Axis of Interest (en el orden de aparición de sus plot spans)
  1. JOURNEY
  2. RIVALRY
  3. CALL TO ACTION REWARD
  4. REPENTANCE
  5. PURSUIT

- Orden en que se entrelazan los distintos Plot Spans (según la narración):
  1) JOURNEY.Out (Departure)
  2) JOURNEY.Back (Return)
  3) RIVALRY.RivalryDeclared
  4) RIVALRY.Cooperation
  5) RIVALRY.Reconciliation
  6) CALL TO ACTION REWARD.Call
  7) CALL TO ACTION REWARD.Reward
  8) REPENTANCE.Transformation
  9) REPENTANCE.Repentance
  10) REPENTANCE.RepentanceRewarded
  11) PURSUIT.Chase
  12) PURSUIT.Escape

Si quieres, puedo darte una representación visual simple (líneas de tiempo) o un esquema JSON con las relaciones entre plot spans.
